## Introductory notebook showing Oceanigans basic high-level abstractions

This notebook looks at a simple problem of stepping forward in time a 1d diffusion equation 

 $$ \frac{\partial T}{\partial t} = \kappa \frac{\partial^2 T}{\partial z^2} $$

on a discrete mesh using Oceananigans abstractions.

The example uses the following three key abstractions

* `grid` 
     - a grid is a three dimensional space that has a logically cartesian index space. A grid index space has a topology that defines the edge rules for limit indices. A grid also has physical space coordinates attached to the index space locations. These coordinates specify the grid's discrete, structured finite volume mesh.
  
  

* `model`
     - a model supports a particular set of discretized in space and time equation evaluation code that is available to be computed on a grid. When a model is defined some or all of the equations are selected as active terms to be integrated.
     
     

* `simulation`
     - a simulation combines a `grid`, a `model` and simulation paramteres. A simulation evaluates the active terms of a model over the discrete space define by a grid.

In [ ]:
using Pkg
pkg"add Oceananigans, CairoMakie"

In [ ]:
using Oceananigans

#### Define a grid instance 
This example defines a 1d index space along the last dimension of the grid with an implied wall at either end (`Bounded` topology). The other two dimensions are tagged as singleton dimensions (`Flat` topology). The final dimension of a gris is labelled `z`. 

In [ ]:
grid = RectilinearGrid(size=128, z=(-0.5, 0.5), topology=(Flat, Flat, Bounded))

#### Get help on full properties of `RectilinearGrid`

In [ ]:
?RectilinearGrid

#### Define a model instance
This example uses a a `NonhydrostaticModel` set of discrete equations. The full model equations contains multiple [terms](https://clima.github.io/OceananigansDocumentation/stable/physics/nonhydrostatic_model/#Nonhydrostatic-model) that solve an entire range of fluid problems. Here we create an instance in which only a single scalar field (`T`) is passed in along with a simple `ScalarDiffusivity` closure. This has the effect of reducing the specific model instance to only solving a simple diffusion equation.

In [ ]:
closure = ScalarDiffusivity(κ=1)
model = NonhydrostaticModel(; grid, closure, tracers=:T)

In [ ]:
?NonhydrostaticModel

In [ ]:
width = 0.1
initial_temperature(x, y, z) = exp(-z^2 / (2width^2))
set!(model, T=initial_temperature)

In [ ]:
using CairoMakie
set_theme!(Theme(fontsize = 24, linewidth=3))

fig = Figure()
axis = (xlabel = "Temperature (ᵒC)", ylabel = "z")
label = "t = 0"

z = znodes(model.tracers.T)
T0 = copy( interior(model.tracers.T, 1, 1, :) )

lines(T0, z; label, axis)

In [ ]:
# Time-scale for diffusion across a grid cell
diffusion_time_scale = model.grid.Δzᵃᵃᶜ^2 / model.closure.κ.T

simulation = Simulation(model, Δt = 0.1 * diffusion_time_scale, stop_iteration = 1000)

In [ ]:
run!(simulation)

In [ ]:
using Printf

set_theme!(Theme(fontsize = 24, linewidth=3))

fig = Figure()
axis = (xlabel = "Temperature (ᵒC)", ylabel = "z")
label = "t = 0"

z = znodes(model.tracers.T)
T = interior(model.tracers.T, 1, 1, :)

lines(T0, z; label, axis)
lines!(T, z; label)
current_figure()

In [ ]:
model.velocities

In [ ]:
simulation.output_writers[:temperature] =
    JLD2OutputWriter(model, model.tracers,
                     filename = "one_dimensional_diffusion.jld2",
                     schedule=IterationInterval(100),
                     overwrite_existing = true)

In [ ]:
simulation.stop_iteration += 10000
run!(simulation)

In [ ]:
T_timeseries = FieldTimeSeries("one_dimensional_diffusion.jld2", "T")
times = T_timeseries.times

fig = Figure()
ax = Axis(fig[2, 1]; xlabel = "Temperature (ᵒC)", ylabel = "z")
xlims!(ax, 0, 1)

n = Observable(1)

T = @lift interior(T_timeseries[$n], 1, 1, :)
lines!(T, z)

label = @lift "t = " * string(round(times[$n], digits=3))
Label(fig[1, 1], label, tellwidth=false)
current_figure()

In [ ]:
frames = 1:length(times)

@info "Making an animation..."

record(fig, "one_dimensional_diffusion.mp4", frames, framerate=24) do i
    msg = string("Plotting frame ", i, " of ", frames[end])
    print(msg * " \r")
    n[] = i
end

In [ ]:
wireframe(Sphere())
